In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
#load
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')
gender_sub=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

In [ ]:
#Understanding Data
#asking q
#data type of colms
print(train.info())
print("\n")
#how data look mathematicaly
print(train.describe())
print("\n")
# how big is the data 
print(train.shape)
print("\n")
#data look like
print(train.head())
print("\n")
print("\n",train.sample)
print("\n")
#missing values
missing_value=train.isnull().sum()
print(missing_value[missing_value>0])
print("\n")
# are there duplicate value 
print(train.duplicated().sum())
print("\n")
#correlation btw colms
print(train.corr(numeric_only=True))



In [ ]:
sns.countplot(data=train,x='Survived')
plt.xticks([0,1],['Died(0)','Survived(1)'])
plt.xlabel('survival status')
plt.title("Survival count")
plt.show()

In [ ]:
#three filters 
#Unique ID if unique drop 
#Missing Data if data mising drop 
#missing values
missing_value=train.isnull().sum()
print("\n missing values",missing_value[missing_value>0])
#Correlation
print("\n",train.nunique())
#If a column has a small number of unique values (like Sex=2, Embarked=3, Pclass=3), you must plot it against survival.

In [ ]:
#now plot  ['Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch']
cat_cols=["Sex","Pclass","Embarked","SibSp","Parch"]
for col in cat_cols:
    plt.figure(figsize=(5,3))
    sns.barplot(x=col,y="Survived",data=train)
    plt.title(f"Survival Rate by {col}")
    plt.show()

In [ ]:
#distribution 
num_cols = ['Age', 'Fare']
for col in num_cols:
    plt.figure(figsize=(5,3))
    sns.histplot(train[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.show()

In [ ]:
# feature engineering
#feature construction 
train['family_size']=train['SibSp']+train['Parch']+1
test['family_size']=test['SibSp']+test['Parch']+1
#feature selection 
col_drop=['PassengerId','Name','Ticket','Cabin','SibSp','Parch']
train.drop(columns=col_drop,inplace=True)
test.drop(columns=col_drop,inplace=True)
print(train.head())

In [ ]:
#For Numerical: Handle Missing Values (Day 26: Simple Imputer) Scale the data (Day 20: Standardization).
#For Categorical: Handle Missing Values (Day 27: Categorical Impute) Convert text to numbers (Day 23: One Hot Encoding).
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
# 1. Numerical Pipeline (Impute + Scale)
num_pipeline=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])
# 2. Categorical Pipeline (Impute + Encode)
cat_pipeline= Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ohe',OneHotEncoder(handle_unknown='ignore'))
])
## 3. Combine them using Column Transformer (Day 24)
preprocessor=ColumnTransformer(transformers=[
    ('num',num_pipeline,['Age','Fare','family_size']),
    ('cat',cat_pipeline,['Sex','Embarked','Pclass'])
])
print("chala diya")

In [ ]:
#Applying the Transformation
X=train.drop('Survived',axis=1)
y=train['Survived']
X_transformed=preprocessor.fit_transform(X)
print("shape of transformed data:",X_transformed.shape)
print("transformation complted !")

In [ ]:
from sklearn.model_selection import train_test_split
X=train.drop(columns=['Survived'])
y=train['Survived']
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=42)
print("done")

In [ ]:
#Train the Model
#Logistic Regression 
from sklearn.linear_model import LogisticRegression
lr_pipeline=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',LogisticRegression())
    
])
#train
lr_pipeline.fit(X_train,y_train)
print("Logistic regresion accuracy :",lr_pipeline.score(X_val,y_val))
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt_pipeline=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',DecisionTreeClassifier(max_depth=5))
])
#train
dt_pipeline.fit(X_train,y_train)
print("decision tree accuracy:",dt_pipeline.score(X_val,y_val))


In [ ]:
#Make Predictions on Kaggle's Test Data
final_prediction_lr=lr_pipeline.predict(test)
final_prediction_dt=dt_pipeline.predict(test)
print("\nPrediction on test by lr\n",final_prediction_lr)
print("\nPredictions on test by dt\n",final_prediction_dt)

In [ ]:
test_data=pd.read_csv('/kaggle/input/titanic/test.csv')
submission=pd.DataFrame({
    "PassengerId":test_data["PassengerId"],
    "Survived":final_prediction_dt
})
print(submission.head())
submission.to_csv('submission.csv',index=False)
print("done")
